In [1]:
import os
import langchain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, ConversationChain
from langchain.callbacks import get_openai_callback
from langchain.memory import ConversationBufferWindowMemory, CombinedMemory, ConversationSummaryMemory, ChatMessageHistory
from langchain import hub
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.ai import AIMessage
from langchain_community.utilities.google_search import GoogleSearchAPIWrapper
from langchain_community.utilities.openweathermap import OpenWeatherMapAPIWrapper
from langchain.agents import AgentExecutor, create_react_agent, Tool
from os import environ
from dotenv import load_dotenv

load_dotenv(dotenv_path=f"{os.path.abspath('')}/../../backend/.env.dev", verbose=True)

True

In [2]:
# parent class

from abc import ABC, abstractmethod


class Agent(ABC):
    name = "Default Agent"

    @abstractmethod
    def load_prompt_template(self, *args, **kwargs):
        print("Load prompt template method not implemented")
        raise NotImplementedError

    @abstractmethod
    def load_tools(self, tool_list):
        print("Load tools method not implemented")
        raise NotImplementedError

    @abstractmethod
    def load_conversation(self, *args, **kwargs):
        print("Load memory method not implemented")
        raise NotImplementedError

    @abstractmethod
    def load_model(self, *args, **kwargs):
        print("Load model method not implemented")
        raise NotImplementedError

    @abstractmethod
    def generate_response(self, user_input):
        print("Generate response method not implemented")
        raise NotImplementedError


In [3]:
# tools

from langchain.agents import Tool
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_community.utilities.openweathermap import OpenWeatherMapAPIWrapper

weather_search_util = OpenWeatherMapAPIWrapper(openweathermap_api_key=os.environ["OPENWEATHERMAP_API_KEY"])
google_search_util = GoogleSearchAPIWrapper(google_api_key=os.environ["GOOGLE_API_KEY"], google_cse_id=os.environ["GOOGLE_CSE_ID"], k=5)


def get_search_google_tool():
    return Tool(
        name="search_google",
        description="Search Google using a query and get a list of results. Recommended to use the language related to the context of the conversation to get the best results.",
        func=google_search_util.run)


def get_search_weather_tool():
    return Tool(
        name="search_weather",
        description="Get current weather information of specific location. The input format is 'City, Country'.",
        func=weather_search_util.run)



In [4]:
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

members = ["Researcher", "Planner", "Reply Agent"]

system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

supervisor_chain = (
        prompt
        | llm.bind_functions(functions=[function_def], function_call="route")
        | JsonOutputFunctionsParser()
)